In [1]:
import tensorflow.keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Input, GlobalMaxPool1D, Activation
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import os 

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
filename = '../dataset_text/miditokens_waitFix.txt'
with open(filename) as f:
    miditokens = f.readlines()
    
midiTokensWithMeta = [tokens.strip().split(' ') for tokens in miditokens]

miditokens = []
for i in range(len(midiTokensWithMeta)):
    if 'timesig:4/4' in midiTokensWithMeta[i]:
        midiInfo = midiTokensWithMeta[i]
        
        miditokens.append(midiTokensWithMeta[i])

In [4]:
tokenizer = Tokenizer() # token -> int
tokenizer.fit_on_texts(miditokens)

In [5]:
#sample = miditokens[1]
#print(sample)
#print(tokenizer.texts_to_sequences([sample])[0]) # Example of token input -> int output
#print("Sõna sagedus kogu andmestikus", tokenizer.word_counts['wait:0.0'])
#print("Sõna -> indeks teisendus", tokenizer.word_index['wait:0.0'])
#print("Indeks -> sõna teisendus", tokenizer.index_word[55])
#print(len(tokenizer.word_index))
#print(tokenizer.index_word)
#print(tokenizer.word_index)

In [6]:
# Turn all tokens to ints
midiTokensAsInt = tokenizer.texts_to_sequences(miditokens)
#midiTokensAsInt = np.array(midiTokensAsInt)

# Generator variant

In [7]:
## https://medium.com/analytics-vidhya/train-keras-model-with-large-dataset-batch-training-6b3099fdf366
#
#def batchGenerator(trainData, VOCAB_SIZE, SEQ_LEN=30):
#    i = 0
#    while True:
#        yield loadDataBatch(trainData, VOCAB_SIZE, SEQ_LEN, i)
#        i += 1
#    
#            
#def loadDataBatch(trainData, VOCAB_SIZE, SEQ_LEN, i):
#    X = []
#    y = []
#    
#    # https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5
#    
#    song = trainData[i]
#    for i in range(0, len(song) - SEQ_LEN, 1):
#        X.append(song[i:i + SEQ_LEN])
#        y.append(song[i + SEQ_LEN])
#            
#    input_len = len(X)
#    
#    # reshape the input into a format compatible with LSTM layers
#    X = np.reshape(X, (input_len, SEQ_LEN, 1))
#    
#    # normalize input
#    X = X / float(VOCAB_SIZE)
#    y = to_categorical(y, num_classes = VOCAB_SIZE)
#    
#    return (X, y)
#
#VOCAB_SIZE = len(tokenizer.word_index)
#SEQ_LEN = 20
#
#batchGen = batchGenerator(midiTokensAsInt, VOCAB_SIZE, SEQ_LEN)

In [8]:
#model = Sequential()
#model.add(LSTM(
#    256,
#    input_shape=(20, 1),
#    return_sequences=True
#))
#model.add(Dropout(0.3))
#model.add(LSTM(256, return_sequences=True))
#model.add(Dropout(0.3))
#model.add(LSTM(256))
#model.add(Dense(256))
#model.add(Dropout(0.3))
#model.add(Dense(VOCAB_SIZE))
#model.add(Activation('softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
#
#filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
#checkpoint = ModelCheckpoint(
#    filepath, monitor='loss', 
#    verbose=0,        
#    save_best_only=True,        
#    mode='min'
#)    
#callbacks_list = [checkpoint]     
##model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)
#
#model.fit(batchGen, 
#          epochs=200,
#          callbacks=callbacks_list,
#          verbose=1)

# Non-generator variant

In [54]:
X = []
y = []

VOCAB_SIZE = len(tokenizer.word_index)
SEQ_LEN = 50

# https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5

for song in midiTokensAsInt[:150]:
    for i in range(0, len(song) - SEQ_LEN, 1):
        X.append(song[i:i + SEQ_LEN])
        y.append(song[i + SEQ_LEN])

X_train = X
y_train = y

input_len = len(X_train)

# reshape the input into a format compatible with LSTM layers
X_train = np.reshape(X_train, (input_len, SEQ_LEN, 1))

# normalize input
#X_train = X_train / float(VOCAB_SIZE)
X_train = to_categorical(X_train, num_classes=VOCAB_SIZE)

y_train = to_categorical(y_train, num_classes=VOCAB_SIZE)

In [55]:
idx = 0
print(X_train[idx], y_train[idx])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [56]:
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(X_train.shape[1], X_train.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(VOCAB_SIZE))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

filepath = "../weights/weights-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(X_train, y_train, epochs=200, batch_size=128, callbacks=callbacks_list)

Epoch 1/200
2721/2721 [==============================] - 135s 47ms/step - loss: 3.9565
Epoch 2/200
2721/2721 [==============================] - 127s 47ms/step - loss: 2.4365
Epoch 3/200
2721/2721 [==============================] - 130s 48ms/step - loss: 1.9565
Epoch 4/200
2721/2721 [==============================] - 130s 48ms/step - loss: 1.7272
Epoch 5/200
2721/2721 [==============================] - 132s 48ms/step - loss: 1.5703
Epoch 6/200
2721/2721 [==============================] - 133s 49ms/step - loss: 1.4583
Epoch 7/200
2721/2721 [==============================] - 129s 47ms/step - loss: 1.3666
Epoch 8/200
2721/2721 [==============================] - 130s 48ms/step - loss: 1.2953
Epoch 9/200
2721/2721 [==============================] - 127s 47ms/step - loss: 1.2333
Epoch 10/200
2721/2721 [==============================] - 129s 47ms/step - loss: 1.1782
Epoch 11/200
2721/2721 [==============================] - 127s 47ms/step - loss: 1.1371
Epoch 12/200
2721/2721 [=================

2721/2721 [==============================] - 135s 50ms/step - loss: 0.5043
Epoch 93/200
2721/2721 [==============================] - 150s 55ms/step - loss: 0.5019
Epoch 94/200
2721/2721 [==============================] - 136s 50ms/step - loss: 0.5010
Epoch 95/200
2721/2721 [==============================] - 130s 48ms/step - loss: 0.4987
Epoch 96/200
2721/2721 [==============================] - 128s 47ms/step - loss: 0.4995
Epoch 97/200
2721/2721 [==============================] - 130s 48ms/step - loss: 0.4916
Epoch 98/200
2721/2721 [==============================] - 131s 48ms/step - loss: 0.4900
Epoch 99/200
2721/2721 [==============================] - 128s 47ms/step - loss: 0.4888
Epoch 100/200
2721/2721 [==============================] - 128s 47ms/step - loss: 0.4866
Epoch 101/200
2721/2721 [==============================] - 132s 48ms/step - loss: 0.4823
Epoch 102/200
2721/2721 [==============================] - 148s 54ms/step - loss: 0.4833
Epoch 103/200
2721/2721 [=================

2721/2721 [==============================] - 139s 51ms/step - loss: 0.3682
Epoch 184/200
2721/2721 [==============================] - 137s 51ms/step - loss: 0.3634
Epoch 185/200
2721/2721 [==============================] - 128s 47ms/step - loss: 0.3653
Epoch 186/200
2721/2721 [==============================] - 127s 47ms/step - loss: 0.3633
Epoch 187/200
2721/2721 [==============================] - 135s 50ms/step - loss: 0.3639
Epoch 188/200
2721/2721 [==============================] - 182s 67ms/step - loss: 0.3659
Epoch 189/200
2721/2721 [==============================] - 143s 53ms/step - loss: 0.3615
Epoch 190/200
2721/2721 [==============================] - 138s 51ms/step - loss: 0.3586
Epoch 191/200
2721/2721 [==============================] - 135s 49ms/step - loss: 0.3561
Epoch 192/200
2721/2721 [==============================] - 143s 52ms/step - loss: 0.3601
Epoch 193/200
2721/2721 [==============================] - 130s 48ms/step - loss: 0.3532
Epoch 194/200
2721/2721 [==========

In [57]:
model.save("LSTM256-onehot.h5")

In [12]:
new_model = tf.keras.models.load_model('LSTM256.h5')

In [10]:
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(X_train.shape[1], X_train.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(VOCAB_SIZE))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# Load the weights to each node
model.load_weights('../weights/weights.hdf5')

In [219]:
start = np.random.randint(0, len(X)-1)
intToNote = dict(map(reversed, tokenizer.word_index.items()))

nrOfSong = 463
pattern = [np.argmax(X_train[start], axis=1)]
pattern = [np.array(midiTokensAsInt[nrOfSong][:50])]
print(pattern)
#pattern = [i for [i] in pattern]
prediction_output = [intToNote[i] for i in list(pattern[0])]

text2midi(prediction_output).show("midi")

#prediction_output = []
#nrOfSong = 42
#pattern = midiTokensAsInt[nrOfSong][:SEQ_LEN]
#for tokenInt in pattern:
#    prediction_output.append(intToNote[tokenInt])

# generate 500 notes
for note_index in range(500):
    #prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    #prediction_input = prediction_input / float(VOCAB_SIZE)
    prediction_input = to_categorical(pattern, num_classes=VOCAB_SIZE)
    
    prediction = model.predict(prediction_input, verbose=0)
    
    index = np.argmax(prediction)
    result = intToNote[index]
    prediction_output.append(result)
    
    #print(index)
    pattern = np.append(pattern, index)
    #print(pattern)
    #print(pattern[1:len(pattern)])
    pattern = [pattern[1:len(pattern)]]
    
if 'end' in prediction_output:
    print('warn')

[array([150, 217, 151,  26,  60,  97,   1,  61,  85,   1,  86,  62,   1,
        63, 119,   1, 120,  60,   1,  61,  85,   1,  86,  62,   1,  63,
       119,   1, 120,  60,   1,  61,  85,   1,  86,  62,   1,  63, 119,
         1, 120,  60,   1,  61,  85,   1,  86,  62,   1,  63])]


In [223]:
print("Whole song:")
text2midi(miditokens[nrOfSong]).show("midi")
print("Input to network:")
text2midi(miditokens[nrOfSong][:SEQ_LEN]).show("midi")

Whole song:


Input to network:


In [221]:
#print(prediction_output)
genmidistream = text2midi(prediction_output)
genmidistream.show("midi")

In [208]:
fp = genmidistream.write('midi', fp='wow2.mid')

In [224]:
from music21 import *

def text2midi(tokens):
    s = stream.Stream()
    
    currentVelocity = 80

    currentOffset = 0
    currentToken = 0

    for token in tokens:

        splitToken = token.split(":")

        if token.startswith("tempo"):
            s.append(tempo.MetronomeMark(number=float(splitToken[1])))

        if token.startswith("timesig"):
            s.append(meter.TimeSignature(splitToken[1]))
            
        if token.startswith("velocity"):
            currentVelocity = int(splitToken[1])

        if token.startswith("note") and not token.lower().endswith("off"):
            noteDuration = 0
            noteName = splitToken[1]

            for element in tokens[currentToken+1:]:
                splitToken2 = element.split(":")
                if (element.startswith("wait")):
                    noteDuration += float(splitToken2[1])
                if (element.startswith("note") and element.lower().endswith("off")):
                    if (noteName == splitToken2[1]):
                        newNote = note.Note(nameWithOctave=splitToken[1],  
                               quarterLength=float(noteDuration))
                        newNote.volume.velocity = currentVelocity
                        s.insert(currentOffset, newNote)
                        break

        if token.startswith("wait"):
            currentOffset += float(splitToken[1]) 

        currentToken += 1

    return s

#text2midi(midiTokens).show("midi")